In [1]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from CBAWL import CBAWL

In [7]:
dataset_id = 101

cbawl = CBAWL(dataset_id, min_support=0.01, min_confidence=0.6, min_lift=0.9)
data = cbawl.dataHandler.loadData()
features = cbawl.dataHandler.oneHotEncoding(data)
features_importance = cbawl.dataHandler.getFeaturesImportance(data)
features = cbawl.dataHandler.delLowImportanceFeatures(data, features_importance)
X = features
y = data.iloc[:, -1]
kf = KFold(n_splits=10, shuffle=True, random_state=42)
precision = []
recall = []
f1 = []
accuracy = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    X_train_te = cbawl.dataHandler.oneHotEncoding(X_train)
    frequent_itemsets = cbawl.ruleGenerator.getFrequentItemsets(X_train_te)
    strong_rules, weak_rules, default_class = cbawl.model(frequent_itemsets, X_train)
    y_pred = cbawl.predict(strong_rules, weak_rules, default_class, X_test)

    precision.append(precision_score(y_test, y_pred, average='macro', zero_division=1))
    recall.append(recall_score(y_test, y_pred, average='macro', zero_division=1))
    f1.append(f1_score(y_test, y_pred, average='macro', zero_division=1))
    accuracy.append(accuracy_score(y_test, y_pred))


print('Precision:', sum(precision)/len(precision))
print('Recall:', sum(recall)/len(recall))
print('F1:', sum(f1)/len(f1))
print('Accuracy:', sum(accuracy)/len(accuracy))

Precision: 0.8297656729761946
Recall: 0.5147254240324892
F1: 0.42370595861795807
Accuracy: 0.6618092105263158
